# Network X Model

## DTM 생성

In [ ]:
import pandas as pd
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
# DTM을 편리하게 만들어주기 위해 Scikit-Learn에서 제공하는 CountVectorizer를 import 한다.


def makeDTM(category):
    # 타이틀 리스트를 불러와서 title_list 변수에 저장한다.
    t_file_name = open('./Title_List.txt', 'r', encoding='utf-8')

    title_list = []
    for line in t_file_name.readlines():
        # txt파일을 readlines로 불러오면 개행 문자도 함께 읽어오기 때문에 인덱싱으로 처리해준다.
        title_list.append(line[:-1])

    t_file_name.close()

    # pandas의 read_csv 함수를 이용하여 csv 파일을 불러온다.
    dataset = pd.read_csv(f'./{category}_crwl.csv')

    # 각 형태소별로 분류(Tagging)해주는 Okt 객체를 불러온다.
    tagger = Okt()

    for title in title_list:        # title_list에 대해 반복문을 실행
        # 각 타이틀에 대한 6770개 문서의 DTM을 표현하기 위해
        # CountVectorizer 객체를 선언
        cv = CountVectorizer()
        
        # 각 문서들의 말뭉치(corpus)를 저장할 리스트 선언
        corpus = []

        # 각 타이틀에 대한 문서들의 말 뭉치를 저장한다. (데이터가 많으면 이 부분에서 장시간이 소요될 수 있다.)
        for doc_num in range(len(dataset)):
            # 각 말뭉치에서 명사 리스트를 만든다.
            noun_list = tagger.nouns(dataset[title].loc[doc_num])
            
            # 이를 문자열로 저장해야하기 때문에 join함수로 공백으로 구분해 corpus에 append한다.
            corpus.append(' '.join(noun_list))

        # CountVectorizer의 fit_transform 함수를 통해 DTM을 한번에 생성할 수 있다.
        DTM_Array = cv.fit_transform(corpus).toarray()

        # feature_names 함수를 사용하면 DTM의 각 열(column)이 어떤 단어에 해당하는지 알 수 있다.
        feature_names = cv.get_feature_names()

        # 추출해낸 데이터를 DataFrame 형식으로 변환한다.
        DTM_DataFrmae = pd.DataFrame(DTM_Array, columns=feature_names)

        # 최종적으로 DTM을 csv 파일로 저장한다.
        DTM_DataFrmae.to_csv('DTM.csv', encoding='utf-8-sig')

## StopWord delete and preprocess

In [ ]:
from tqdm import tqdm
def NLP_DTM(category):
    # 타이틀 리스트를 불러와서 title_list 변수에 저장한다.
    t_file_name = open('./Title_List.txt', 'r', encoding='utf-8')

    title_list = []
    for line in t_file_name.readlines():
        # txt파일을 readlines로 불러오면 개행 문자도 함께 읽어오기 때문에 인덱싱으로 처리해준다.
        title_list.append(line[:-1])

    t_file_name.close()

    # 불용어 파일을 불러와서 stop_words_list 변수에 저장한다.
    s_file_name = open('./koreanStopwords.txt', 'r', encoding='utf-8')

    stop_words_list = []
    for line in s_file_name.readlines():
        # 위에서는 인덱싱으로 처리했지만 rstrip 함수를 사용하여 공백을 제거할 수도 있다.
        stop_words_list.append(line.rstrip())

    s_file_name.close()

    # pandas의 read_csv 함수를 이용하여 csv 파일을 불러온다.
    dataset = pd.read_csv(f'./{category}_crwl.csv')

    # 각 형태소별로 분류(Tagging)해주는 Okt 객체를 불러온다.
    tagger = Okt()

    for title in tqdm(title_list, desc='타이틀 리스트 진행도'):  # title_list에 대해 반복문을 실행
        # 각 타이틀에 대한 6770개 문서의 DTM을 표현하기 위해
        # CountVectorizer 객체를 선언
        cv = CountVectorizer()

        # 각 문서들의 말뭉치(corpus)를 저장할 리스트 선언
        corpus = []

        # 각 타이틀에 대한 문서들의 말 뭉치를 저장한다. (데이터가 많으면 이 부분에서 장시간이 소요될 수 있다.)
        for doc_num in tqdm(range(len(dataset)), desc='문서 진행도'):
            # 각 말뭉치에서 명사 리스트를 만든다.
            noun_list = tagger.nouns(dataset[title].loc[doc_num])

            # 이를 문자열로 저장해야하기 때문에 join함수로 공백으로 구분해 corpus에 append한다.
            corpus.append(' '.join(noun_list))

        # CountVectorizer의 fit_transform 함수를 통해 DTM을 한번에 생성할 수 있다.
        DTM_Array = cv.fit_transform(corpus).toarray()

        # feature_names 함수를 사용하면 DTM의 각 열(column)이 어떤 단어에 해당하는지 알 수 있다.
        feature_names = cv.get_feature_names()

        # 추출해낸 데이터를 DataFrame 형식으로 변환한다.
        DTM_DataFrmae = pd.DataFrame(DTM_Array, columns=feature_names)

        # 열 제거는 drop 함수를 사용하며 axis 속성을 columns로 준 후 inplace를 True로 한다.
        del_col = set(DTM_DataFrmae.columns).intersection(set(stop_words_list))
        DTM_DataFrmae.drop(del_col, axis='columns', inplace=True)

        # 최종적으로 DTM을 csv 파일로 저장한다.
        DTM_DataFrmae.to_csv('DTM.csv', encoding='utf-8-sig')



## co-occurence network anlysis

In [ ]:
# -*- encoding: utf-8 -*-
import pandas as pd
from tqdm import tqdm

def networkX():
    # 단어쌍의 빈도를 체크하기위해 DTM을 불러온다.
    dataset = pd.read_csv('DTM.csv')

    # 단어들의 목록을 가져온다.
    # 이때 0번째 인덱스에는 빈 칸이 들어오므로 인덱싱을 통해 없애준다.
    column_list = dataset.columns[1:]
    word_length = len(column_list)

    # 각 단어쌍의 빈도수를 저장할 dictionary 생성
    count_dict = {}

    for doc_number in tqdm(range(len(dataset)), desc='단어쌍 만들기 진행중'):
        tmp = dataset.loc[doc_number]           # 현재 문서의 단어 출현 빈도 데이터를 가져온다.
        for i, word1 in enumerate(column_list):
            if tmp[word1]:              # 현재 문서에 첫번째 단어가 존재할 경우
                for j in range(i + 1, word_length):
                    if tmp[column_list[j]]:              # 현재 문서에 두번째 단어가 존재할 경우
                        count_dict[column_list[i], column_list[j]] = count_dict.get((column_list[i], column_list[j]), 0) + max(tmp[word1], tmp[column_list[j]])

    # count_list에 word1, word2, frequency 형태로 저장할 것이다.
    count_list = []

    for words in count_dict:
        count_list.append([words[0], words[1], count_dict[words]])

    # 단어쌍 동시 출현 빈도를 DataFrame 형식으로 만든다.
    df = pd.DataFrame(count_list, columns=["word1", "word2", "freq"])
    df = df.sort_values(by=['freq'], ascending=False)
    df = df.reset_index(drop=True)

    # 이 작업이 오래 걸리기 때문에 csv파일로 저장 후 사용하는 것을 추천한다.
    df.to_csv('networkx.csv', encoding='utf-8-sig')

## Network X Graph

In [ ]:
import pandas as pd
import networkx as nx
import operator
import numpy as np

def show(show_num,category,iteration):
    # 단어쌍 동시출현 빈도수를 담았던 networkx.csv파일을 불러온다.
    dataset = pd.read_csv('./networkx.csv')

    # 중심성 척도 계산을 위한 Graph를 만든다
    G_centrality = nx.Graph()

    # 빈도수가 20000 이상인 단어쌍에 대해서만 edge(간선)을 표현한다.
    for ind in range(len(np.where(dataset['freq'])[0][:show_num])):
        G_centrality.add_edge(dataset['word1'][ind], dataset['word2'][ind], weight=int(dataset['freq'][ind]))

    dgr = nx.degree_centrality(G_centrality)        # 연결 중심성
#     btw = nx.betweenness_centrality(G_centrality)   # 매개 중심성
#     cls2 = nx.closeness_centrality(G_centrality)     # 근접 중심성
#     egv = nx.eigenvector_centrality(G_centrality)   # 고유벡터 중심성
    pgr = nx.pagerank(G_centrality)                 # 페이지 랭크

    # 중심성이 큰 순서대로 정렬한다.
    sorted_dgr = sorted(dgr.items(), key=operator.itemgetter(1), reverse=True)
#     sorted_btw = sorted(btw.items(), key=operator.itemgetter(1), reverse=True)
#     sorted_cls = sorted(cls2.items(), key=operator.itemgetter(1), reverse=True)
#     sorted_egv = sorted(egv.items(), key=operator.itemgetter(1), reverse=True)
    sorted_pgr = sorted(pgr.items(), key=operator.itemgetter(1), reverse=True)

    # 단어 네트워크를 그려줄 Graph 선언
    G = nx.Graph()

    # 페이지 랭크에 따라 두 노드 사이의 연관성을 결정한다. (단어쌍의 연관성)
    # 연결 중심성으로 계산한 척도에 따라 노드의 크기가 결정된다. (단어의 등장 빈도수)
    for i in range(len(sorted_pgr)):
        G.add_node(sorted_pgr[i][0], nodesize=sorted_dgr[i][1])

    for ind in range(len(np.where(dataset['freq'])[0][:show_num])):
        G.add_weighted_edges_from([(dataset['word1'][ind], dataset['word2'][ind], int(dataset['freq'][ind]))])


    # 노드 크기 조정
    sizes = [G.nodes[node]['nodesize'] * 1000 for node in G]

    options = {
        'edge_color': '#bfc3f5',
        'width': 1.5,
        'with_labels': True,
        'font_weight': 'bold',
        'font_size':11,
    }

    # 폰트 설정을 위한 font_manager import
    import matplotlib.font_manager as fm
    import matplotlib.pyplot as plt

    # 폰트 설정

    font_fname = '/usr/share/fonts/nanum/NanumMyeongjo.ttf'      
    fontprop = fm.FontProperties(fname=font_fname, size=100).get_name()

    fig, ax = plt.subplots()
    nx.draw(G, node_size=sizes, pos=nx.spring_layout(G, k=4, iterations=iteration), node_color="#edabc5", **options, font_family=fontprop)  # font_family로 폰트 등록
#     nx.draw(G, node_size=0,pos=nx.spring_layout(G, k=2.5, iterations=100), **options, font_family=fontprop)  # font_family로 폰트 등록
    ax = plt.gca()
    ax.collections[0].set_edgecolor("#bfc3f5")


    fig.set_facecolor('#ffffff')
    
    plt.savefig(f'{category}_network.png', dpi=600)
    plt.show()
